In [ ]:
# Import necessary libraries
import numpy as np
import librosa
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.fftpack as fft
from scipy.signal import medfilt
import IPython.display as ipd

In [ ]:
# Load the audio file
y, sr = librosa.load('original.wav', sr=None)

# Perform Short-Time Fourier Transform (STFT) and extract magnitude and phase
S_full, phase = librosa.magphase(librosa.stft(y))

# Calculate noise power based on the first 10ms
noise_power = np.mean(S_full[:, :int(sr*.1)], axis = 1)

## Old Binary Mask
# Mask to identify frequncies with power greater than noise power
#mask = S_full > noise_power[:, None]
#mask = mask.astype(float)

## New Soft Masking
# Calculate threshold
threshold = noise_power[:, None] * 0.6
# Steepness of signmoid to adjust how abrupt or gradual the transition will be
steepness = 2
# Soft masking using a sigmoid function
mask = 1 / (1 + np.exp(-steepness * (S_full - threshold)))

# Apply median filtering to smooth te mask and reduce noise
mask = medfilt(mask, kernel_size = (1, 5))

# Apply the mask to magnitude spectogram
S_clean = S_full * mask

# Perform inverse STFT to reconstruct audio signal
y_clean = librosa.istft(S_clean * phase)

#Save cleaned audio to a new file
sf.write('clean.wav', y_clean, sr)

Things to try:
Frequency-Dependent Thresholds or Steepness

Explanation: You can apply different threshold factors or steepness values to different frequency bands. This is useful because noise often varies across frequencies.
How to Adjust:
Divide the spectrogram (S_full) into frequency bands (e.g., using librosa.fft_frequencies to get frequency bins and slicing S_full).
Calculate and apply separate masks with different threshold factors or steepness values for each band.
Combine the masked bands back into a single spectrogram.

In [ ]:
# Play orignal audio
ipd.Audio('/content/original.wav')

Buffered data was truncated after reaching the output size limit.

In [ ]:
#Play cleaned audio
ipd.Audio('/content/clean.wav')